In [ ]:
'''
Test Commands

python main.py C:\Users\user\Documents\fe_assignment\MedGridDesign.txt C:\Users\user\Documents\fe_assignment\MedGridTest.txt
python main.py C:\Users\user\Documents\fe_assignment\SmallGridDesign.txt C:\Users\user\Documents\fe_assignment\SmallGridTest.txt

''' 

In [1]:
# Parameters
length = 8   # Number of holes on each side
mirror_positions = [(3,2),(3,7),(6,4),(8,7,10)]    # Mirror positions
rays = ['C2+','C3+','C5+','C6-','R4+','R5+','C7+','R5-'] # ['C7+','C5+','R5-']

In [25]:
# Medium Grid Parameters
length = 8   # Number of holes on each side
mirror_positions = [(3,2),(3,7),(6,4),(8,7,10)]    # Mirror positions
rays = ['C1+', 'C2+', 'C3+', 'C4+', 'C5+', 'C6+', 'C7+', 
        'C8+', 
        'C1-', 'C2-', 'C3-', 'C4-', 'C5-', 'C6-', 'C7-', 'C8-', 'R1+', 'R2+', 'R3+', 'R4+', 'R5+', 'R6+', 'R7+', 'R8+', 'R1-', 'R2-', 'R3-', 'R4-', 'R5-', 'R6-', 'R7-', 'R8-'
        ]

In [2]:
# Cell Class
class Cell():
    def __init__(self) -> None:
        self.mirror = False
        self.life = float('inf')

In [3]:
def createGrid(length,mirror_positions):
    # Create Empty Grid
    rows = []
    for _ in range(length):
        columns = []
        for _ in range(length):
            columns.append(Cell())
        rows.append(columns)
    print(rows)

    # Insert Mirrors
    for mirror in mirror_positions:
        row_idx = mirror[0] - 1
        col_idx = mirror[1] - 1
        cell = rows[row_idx][col_idx]
        cell.mirror = True
        if len(mirror) == 3:
            cell.life = mirror[2] 
    return rows

In [16]:
# Beam Class
class Beam():
    def __init__(self,start_coordinates,length,grid) -> None:
        self.start_coordinates = start_coordinates
        self.end = False
        # Initialise the starting coordinates of the beam
        if start_coordinates[0]=='R' and start_coordinates[2]=='+':
            self.curr_row_idx = int(start_coordinates[1]) - 1 
            self.curr_col_idx = 0
            self.current_direction = 'right'
        elif start_coordinates[0]=='R' and start_coordinates[2]=='-':
            self.curr_row_idx = int(start_coordinates[1]) - 1
            self.curr_col_idx = length - 1
            self.current_direction = 'left'    
        elif start_coordinates[0]=='C' and start_coordinates[2]=='+':
            self.curr_row_idx = 0
            self.curr_col_idx = int(start_coordinates[1]) - 1
            self.current_direction = 'down'
        elif start_coordinates[0]=='C' and start_coordinates[2]=='-':
            self.curr_row_idx = length - 1
            self.curr_col_idx = int(start_coordinates[1]) - 1
            self.current_direction = 'up'

        print('Original Beam Position:',self.current_direction,self.curr_row_idx+1,self.curr_col_idx+1)

        # If mirror right beside end immediately
        if start_coordinates[0]=='C':
            if not (self.curr_col_idx == 0):
                if grid[self.curr_row_idx][self.curr_col_idx - 1].mirror:
                    self.end = True
                    self.print_output()
            if not (self.curr_col_idx == (length - 1)):
                if grid[self.curr_row_idx][self.curr_col_idx + 1].mirror:
                    self.end = True
                    self.print_output()
        if start_coordinates[0]=='R':
            if not (self.curr_row_idx == 0):
                if grid[self.curr_row_idx - 1][self.curr_col_idx].mirror:
                    self.end = True
                    self.print_output()
            if not (self.curr_row_idx == (length - 1)):
                if grid[self.curr_row_idx + 1][self.curr_col_idx].mirror:
                    self.end = True
                    self.print_output()
    
    def check_collision(self,grid):
        if self.end:
            return None
        # Check if current cell is a mirror
        curr_grid_cell = grid[self.curr_row_idx][self.curr_col_idx]
        if curr_grid_cell.mirror == True:
            self.current_direction = None
            self.end = True
            print(self.start_coordinates,'->')
            curr_grid_cell.life -= 1
            if curr_grid_cell.life == 0:
                curr_grid_cell.mirror = False
                curr_grid_cell.life = float('inf')
            return None

    def check_mirror(self,grid):
        if self.end:
            return None
        top_right_mirror = False
        bottom_right_mirror = False
        top_left_mirror = False
        bottom_left_mirror = False

        # Check whether adjacent diagonals are mirrors and redirect
        if not (self.curr_col_idx == length - 1) and not (self.curr_row_idx == 0):
            top_right_mirror = grid[self.curr_row_idx-1][self.curr_col_idx+1].mirror
        if not (self.curr_col_idx == length - 1) and not (self.curr_row_idx == length - 1):
            bottom_right_mirror = grid[self.curr_row_idx+1][self.curr_col_idx+1].mirror
        if not (self.curr_col_idx == 0) and not (self.curr_row_idx == 0):
            top_left_mirror = grid[self.curr_row_idx-1][self.curr_col_idx-1].mirror
        if not (self.curr_col_idx == 0) and not (self.curr_row_idx == length - 1):
            bottom_left_mirror = grid[self.curr_row_idx+1][self.curr_col_idx-1].mirror

        if self.current_direction  == 'right':
            if top_right_mirror and bottom_right_mirror:
                self.current_direction = 'left'
            elif  top_right_mirror:
                self.current_direction = 'down'
            elif bottom_right_mirror:
                self.current_direction = 'up'

        elif self.current_direction  == 'left':
            if top_left_mirror and bottom_left_mirror:
                self.current_direction = 'right'
            elif top_left_mirror:
                self.current_direction = 'down'
            elif  bottom_left_mirror:
                self.current_direction = 'up'

        elif self.current_direction  == 'up':
            if top_left_mirror and top_right_mirror:
                self.current_direction = 'down'
            elif top_left_mirror:
                self.current_direction = 'right'
            elif top_right_mirror:
                self.current_direction = 'left'

        elif self.current_direction  == 'down':
            if bottom_left_mirror and bottom_right_mirror:
                self.current_direction = 'up'
            elif bottom_left_mirror:
                self.current_direction = 'right'
            elif bottom_right_mirror:
                self.current_direction = 'left'
        
        print("Check current direction",self.current_direction)
    
    def print_output(self):
        print(self.start_coordinates,'->','{'+str(self.curr_row_idx + 1)+','+str(self.curr_col_idx + 1)+'}')

    def check_end(self):
        if self.current_direction == 'up' and self.curr_row_idx == 0:
            self.end = True
            self.print_output()
        elif self.current_direction == 'right' and self.curr_col_idx == (length - 1):
            self.end = True
            self.print_output()
        elif self.current_direction == 'down' and self.curr_row_idx == (length - 1):
            self.end = True
            self.print_output()    
        elif self.current_direction == 'left' and self.curr_col_idx == 0:
            self.end = True
            self.print_output()    
    
    def move(self):
        if self.end:
            return None
        # Move beam one step in current direction
        if self.current_direction  == 'right':
            self.curr_col_idx += 1
        elif self.current_direction  == 'left':
            self.curr_col_idx -= 1
        elif self.current_direction  == 'up':
            self.curr_row_idx -= 1
        elif self.current_direction  == 'down':
            self.curr_row_idx += 1

In [17]:
#rows[5][3].mirror

In [18]:
#not (beam1.curr_col_idx == length - 1)

In [95]:
'''
beam1 = Beam(rays[3],length)
beam1.check_end()
beam1.check_mirror()
beam1.move()
beam1.check_collision()
print(beam1.curr_row_idx+1,beam1.curr_col_idx+1)
'''

7 6


In [22]:
class Session():
    def __init__(self,length,mirror_positions,rays):
        self.length = length
        self.mirror_positions = mirror_positions
        self.rays = rays
        self.grid = createGrid(length,mirror_positions)
    
    def start_simulation(self):
        for ray in rays:
            beam1 = Beam(ray,length,self.grid)
            while not beam1.end:
                beam1.check_mirror(self.grid)
                beam1.check_end()
                beam1.move()
                beam1.check_collision(self.grid)
                print(beam1.curr_row_idx,beam1.curr_col_idx)

In [26]:
session = Session(length,mirror_positions,rays)
session.start_simulation()

[[<__main__.Cell object at 0x000001943C73EC90>, <__main__.Cell object at 0x000001943C73D220>, <__main__.Cell object at 0x000001943C73C950>, <__main__.Cell object at 0x000001943C73DEE0>, <__main__.Cell object at 0x000001943C73DCD0>, <__main__.Cell object at 0x000001943C73EA80>, <__main__.Cell object at 0x000001943C73D5E0>, <__main__.Cell object at 0x000001943C73CDD0>], [<__main__.Cell object at 0x000001943C73D070>, <__main__.Cell object at 0x000001943C73C410>, <__main__.Cell object at 0x000001943C73CFE0>, <__main__.Cell object at 0x000001943C73C680>, <__main__.Cell object at 0x000001943C73FE30>, <__main__.Cell object at 0x000001943C73FB60>, <__main__.Cell object at 0x000001943C73FAD0>, <__main__.Cell object at 0x000001943C73F470>], [<__main__.Cell object at 0x000001943C73CB60>, <__main__.Cell object at 0x000001943C73E960>, <__main__.Cell object at 0x000001943C73D100>, <__main__.Cell object at 0x000001943C73DDF0>, <__main__.Cell object at 0x000001943C73EFC0>, <__main__.Cell object at 0x0

In [15]:
session.grid[1][6].mirror

False

In [74]:
rays[0][0]

'C'

In [110]:
beam1.curr_col_idx

7

In [112]:
print(not (beam1.curr_col_idx == (length - 1)))

False


In [44]:
rows[beam1.curr_row_idx+1][beam1.curr_col_idx-1].mirror

True

In [ ]:
def main():
    """
    Manage application dependencies
    """
    config_logger()
    logger.info("Configured logger")

    consumer = XXX
    conumer.run()


if __main__ == '__main__':
    # parse and verify the input (abstract input validation with another function or file)
    try:
        main()
    except KeyboardInterrupt:
        logger.info('Interrupted')
        try:
            sys.exit()
        except SystemExit:
            os._exit(0)